In [3]:
import json, os, random, pytz, sys
sys.path.append("../clevr-dataset-gen/image_generation")
from render_images import *

In [ ]:
def render_all_images(scene_struct, num_objects, args, camera):
    """
    Add random objects to the current blender scene
    """

    # Load the property file
    with open(args.properties_json, 'r') as f:
        properties = json.load(f)
        color_name_to_rgba = {}
        for name, rgb in properties['colors'].items():
        rgba = [float(c) / 255.0 for c in rgb] + [1.0]
        color_name_to_rgba[name] = rgba
        material_mapping = [(v, k) for k, v in properties['materials'].items()]
        object_mapping = [(v, k) for k, v in properties['shapes'].items()]
        size_mapping = list(properties['sizes'].items())

    color_shape_combos = [(c, o) for c in color_name_to_rgba.items() for o in object_mapping]

    

    for i in range(len(color_shape_combos)):
        for j in range(len(color_shape_combos)):
            if j == i: continue
            combo1, combo2 = color_shape_combos[i], color_shape_combos[j]
            c1, o1 = combo1
            c2, o2 = combo2

            color_name1, rgba1 = c1
            color_name2, rgba2 = c2
            obj_name1, obj_name_out1 = o1
            obj_name2, obj_name_out2 = o2
            
            # Choose a random size
            for s1 in size_mapping:
                for s2 in size_mapping:
                    size_name1, r1 = s1
                    size_name2, r2 = s2

                    x1, x2 = 0, 0
                    y1, y2 = r1, -r2
        
                    # For cube, adjust the size a bit
                    #if obj_name == 'Cube':
                    #  r /= math.sqrt(2)

                    # Choose random orientation for both objects.
                    theta = 360.0 * random.random()

                    # Actually add the object to the scene

                    objects, blender_objects = render_image(
                        [x1, x2], 
                        [y1, y2], 
                        [r1, r2], 
                        [o1, o2],
                        [size_name1, size_name2],
                        [c1, c2]
                    )
                        
                        

  

In [ ]:
def render_image(X, Y, R, Obj, Size, Color, theta):
    positions = []
    objects = []
    blender_objects = []
    for x, y, r, o, s, c in zip(X, Y, R, Obj, Size, Color):
        utils.add_object(args.shape_dir, o[1], r, (x, y), theta=theta)
        obj = bpy.context.object
        blender_objects.append(obj)
        positions.append((x1, y1, r1))

        # Attach a random material
        mat_name, mat_name_out = material_mapping[0] # fix material
        utils.add_material(mat_name, Color=c[1])

        # Record data about the object in the scene data structure
        pixel_coords = utils.get_camera_coords(camera, obj.location)
        objects.append({
            'shape': o[1],
            'size': str(s),
            'material': mat_name_out,
            '3d_coords': tuple(obj.location),
            'rotation': theta,
            'pixel_coords': pixel_coords,
            'color': c,
        })

    # Check that all objects are at least partially visible in the rendered image
    all_visible = check_visibility(blender_objects, args.min_pixels_per_object)
    if not all_visible:
        print('Some objects are occluded; give up')
        return None, None
    return objects, blender_objects